In [2]:
import pandas as pd
import sympy as sy

In [2]:
inputData = pd.read_csv('Methionine Prediction Low Abund.csv')

In [1]:
run calcIsotopologues.py

In [4]:
run solveSystem.py

Both to simulate a measurement and to reconstruct the molecule based on measurements, we begin by defining basic information about the experiment. This is included in the top block of the .csv file. It includes our sites, the number of atoms present ("Stoich"), and how fragments sample the sites. It also includes site-specific delta values that will be used to simulate the measurements. 

In [5]:
############################################################################################################
###  First, initialize the molecule and the fragments we will be looking at. This includes an "IDList",  ###
###  which gives names to the sites, "elIDs", giving their chemical element, "numberAtSite", giving the  ###
###  number of atoms per site, "deltas", giving delta 13C vs PDB, 17O vs VSMOW, 33S vs CDT, 15N vs AIR,  ###
###  and D vs VSMOW. 18O and 34/36S are assigned delta values based on mass scaling laws. The details    ###
###  can be find in the basicDeltaOperations file.                                                       ###
###                                                                                                      ###
###  We then initialize the fragments, giving them entries of 1 or 'x' to indicate sites which are       ###
###  retained or lost. We put all this information into a dataFrame to easily check.                     ###
###                                                                                                      ###
############################################################################################################

#Define the "set of element isotopes" for each chemical element
subsDict = {'H':(0,1),'N':(0,1),'C':(0,1),'O':(0,1,2),'S':(0,1,2,4)}

##### INITIALIZE SITES #####
IDList = ['Cmethyl','Cgamma','Calphabeta','Ccarboxyl','Ocarboxyl1','Ocarboxyl2','Ssulfur','Namine','Hmethyl','Hgamma',
         'Halphabeta','Hamine','Hhydroxyl','Hprotonated']
elIDs = ['C','C','C','C','O','O','S','N','H','H','H','H','H','H']
numberAtSite = [1,1,2,1,1,1,1,1,3,2,3,2,1,1]

deltas = [-45,-35,-30,-25,-13,-13,2.5,10,-250,-100,0,100,250,0]

frag_full = [1,1,1,1,1,1,1,1,1,1,1,1,1,1]
frag_133 = [1,1,1,1,1,1,1,'x',1,1,1,'x',1,'x']
frag_104 = [1,1,1,'x','x','x',1,1,1,1,1,1,'x','x']
frag_102 = ['x',1,1,1,1,1,'x',1,'x',1,1,1,1,'x']
#88 and both 74 are incorrect but right stoich for predictions
#C3H8ON
#C2H4O2N
frag_88 = [1,1,1,'x','x','x',1,'x',1,1,'x',1,'x','x']
frag_74High = [1,'x',1,'x',1,'x','x',1,1,1,1,'x','x','x']
frag_74Low = [1,1,'x','x',1,1,'x',1,'x',1,'x',1,'x','x']
frag_61 = [1,1,'x','x','x','x',1,'x',1,1,'x','x','x','x']
frag_56 = ['x',1,1,'x','x','x','x',1,'x',1,1,'x',1,'x']

l = [elIDs, numberAtSite, deltas, frag_full, frag_133, frag_104, frag_102,
     frag_88, frag_74High, frag_74Low, frag_61, frag_56]

infoDf = pd.DataFrame(l, columns = IDList)
infoDf = infoDf.transpose()
infoDf.columns = ['IDS','Number','deltas','frag_full','frag_133','frag_104','frag 102',
                  'frag_88','frag_74High','frag_74Low','frag 61','frag 56']

In [6]:
df = infoDf
df

,IDS,Number,deltas,frag_full,frag_133,frag_104,frag 102,frag_88,frag_74High,frag_74Low,frag 61,frag 56
Cmethyl,C,1,-45,1,1,1,x,1,1,1,1,x
Cgamma,C,1,-35,1,1,1,1,1,x,1,1,1
Calphabeta,C,2,-30,1,1,1,1,1,1,x,x,1
Ccarboxyl,C,1,-25,1,1,x,1,x,x,x,x,x
Ocarboxyl1,O,1,-13,1,1,x,1,x,1,1,x,x
Ocarboxyl2,O,1,-13,1,1,x,1,x,x,1,x,x
Ssulfur,S,1,2.5,1,1,1,x,1,x,x,1,x
Namine,N,1,10,1,x,1,1,x,1,1,x,1
Hmethyl,H,3,-250,1,1,1,x,1,1,x,1,x
Hgamma,H,2,-100,1,1,1,1,1,1,1,1,1


In [7]:
byCondensed = calcCondensedDictionary(df)

In [8]:
#To improve computational speed, we could calculate just M0, ... MN populations without calculating byCondensed
M0, M1, M2, M3, M4 = massSelections(byCondensed)

In [9]:
M2Measurements = {}

for i, v in inputData.iterrows():
    if len(v['Unnamed: 0'].split(' ')) > 2:
        frag = v['Unnamed: 0'].split(' ')[1]
        sub = v['Unnamed: 0'].split(' ')[2]
        if sub == '':
            sub = 'Unsub'

        if v['Unnamed: 0'][:2] == 'M2':
            if frag not in M2Measurements:
                M2Measurements[frag] = {}
            M2Measurements[frag][sub] = v['Adj. Rel. Abundance']

M2Df = pd.DataFrame.from_dict(M2Measurements)
M2Df.fillna(0, inplace = True)

In [10]:
fragments = [frag_133, frag_104, frag_102, frag_61, frag_56]
fragKeys = ['133','104','102','61','56']

In [11]:
inputData

,Unnamed: 0,Abs. Abundance,Rel. Abundance,Adj. Rel. Abundance
0,13C U Value,0.054332,0.054332,0.054332
1,15N U Value,0.003713,0.003713,0.003713
2,33S U Value,0.007915,0.007915,0.007915
3,34S U Value,0.044959,0.044959,0.044959
4,17O U Value,0.000750,0.000750,0.000750
...,...,...,...,...
189,36S/13C34S,0.043495,0.043495,0.043495
190,13C34S15N/36S,0.085361,0.085361,0.085361
191,13C34SD/36S,0.041183,0.041183,0.041183
192,18O18O/36S,0.035976,0.035976,0.035976


In [12]:
UValueReference = {'M1':{'133':('13C',0.054331862),'104':('33S',0.007915306920),'102':('15N',0.003712760),
                         '61':('33S',0.007915306920),'56':('15N',0.003712760)},
                   #M2 133 should be 34S
                   'M2':{'133':('34S',0.044958744),'104':('34S',0.044958744),
                                 '102':('18O',0.003910140000000),'61':('34S',0.044958744),
                                 '56':None},
                   #M3 102 best is 18O15N
                   'M3':{'133':('13C34S',0.002442692274701),'104':('34S15N',0.000166921026373),
                         '102':None,
                         '61':None,'56':None},
                   'M4':{'133':('18O34S',0.000175794983264 ),'104':('36S',0.000106244973786),
                         '102':('18O18O',0.000003822298705),
                         '61':('36S',0.000106244973786),'56':None}}

In [13]:
UnsubConc = M0['000000000000000000000']['Conc']

siteElements = strSiteElements(df)
#For each fragment we will observe
for j, fragment in enumerate(fragments):
    #compute the isotopologues present after fragmentation and track their concentrations
    fragmentedDict = {}
    for isotopologue, value in M2.items():
        value['Stochastic U'] = value['Conc'] / UnsubConc
        newIsotopologue = expandAndFragment(fragment, df['Number'].values, isotopologue)
        M2[isotopologue][fragKeys[j] + ' Identity'] = newIsotopologue
        
        sub = computeSubs(newIsotopologue, siteElements)
        
        if sub == '':
            sub = 'Unsub'
            
        M2[isotopologue][fragKeys[j] + ' Subs'] = sub

In [14]:
len(M2)

99

In [15]:
M2

{'000000000000000000011': {'Number': 1,
  'full': '00(0, 0)00000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)11',
  'Conc': 2.7009970354498775e-08,
  'Mass': 2,
  'Subs': 'DD',
  'Stochastic U': 3.0326472000000015e-08,
  '133 Identity': '00000000x00000000xx1x',
  '133 Subs': 'D',
  '104 Identity': '0000xxx000000000000xx',
  '104 Subs': 'Unsub',
  '102 Identity': 'x000000x0xxx00000001x',
  '102 Subs': 'D',
  '61 Identity': '00xxxxx0x00000xxxxxxx',
  '61 Subs': 'Unsub',
  '56 Identity': 'x000xxxx0xxx00000xx1x',
  '56 Subs': 'D'},
 '000000000000000000101': {'Number': 2,
  'full': '00(0, 0)00000(0, 0, 0)(0, 0)(0, 0, 0)(0, 1)01',
  'Conc': 4.7537547823917845e-08,
  'Mass': 2,
  'Subs': 'DD',
  'Stochastic U': 5.337459072000002e-08,
  '133 Identity': '00000000x00000000xx0x',
  '133 Subs': 'Unsub',
  '104 Identity': '0000xxx000000000001xx',
  '104 Subs': 'D',
  '102 Identity': 'x000000x0xxx00000010x',
  '102 Subs': 'D',
  '61 Identity': '00xxxxx0x00000xxxxxxx',
  '61 Subs': 'Unsub',
  '56 Identity': 'x000xx

In [16]:
elIDs = df['IDS'].values
numberAtSite = df['Number'].values

siteListN = [(x,y) for x,y in zip(df.index, numberAtSite)]
siteNameList = []
for site in siteListN:
    siteNameList += [site[0]] * site[1]

In [17]:
def filterEmptyStr(string):
    if string == '':
        return False
    else:
        return True
    
Isotopologues = pd.DataFrame.from_dict(M2).T
Isotopologues.rename(columns={'Conc':'Stochastic',"Subs": "Composition"},inplace = True)
preciseStrings = []

for i, v in Isotopologues.iterrows():
    index = list(Isotopologues.index).index(i)
    Subs = [uEl(element, int(number)) for element, number in zip(siteElements, i)]
    Precise = [x + " " + y for x, y in zip(Subs, siteNameList) if x != '']
    output = '   |   '.join(Precise)
    preciseStrings.append(output)
Isotopologues['Precise Identity'] = preciseStrings
Isotopologues.sort_values('Composition',inplace = True)

In [18]:
Isotopologues[Isotopologues['Precise Identity'] == '13C Calphabeta   |   13C Calphabeta']

,Number,full,Stochastic,Mass,Composition,Stochastic U,133 Identity,133 Subs,104 Identity,104 Subs,102 Identity,102 Subs,61 Identity,61 Subs,56 Identity,56 Subs,Precise Identity
001100000000000000000,1,"00(1, 1)00000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.000105819,2,13C13C,0.000118812,00110000x00000000xx0x,13C/13C,0011xxx000000000000xx,13C/13C,x011000x0xxx00000000x,13C/13C,00xxxxx0x00000xxxxxxx,Unsub,x011xxxx0xxx00000xx0x,13C/13C,13C Calphabeta | 13C Calphabeta


In [19]:
pd.set_option("precision", 15)
pd.set_option('display.max_rows', 500)
Isotopologues

,Number,full,Stochastic,Mass,Composition,Stochastic U,133 Identity,133 Subs,104 Identity,104 Subs,102 Identity,102 Subs,61 Identity,61 Subs,56 Identity,56 Subs,Precise Identity
110000000000000000000,1,"11(0, 0)00000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.000103645183182042,2,13C13C,0.000116371573328348,11000000x00000000xx0x,13C/13C,1100xxx000000000000xx,13C/13C,x100000x0xxx00000000x,13C,11xxxxx0x00000xxxxxxx,13C/13C,x100xxxx0xxx00000xx0x,13C,13C Cmethyl | 13C Cgamma
100010000000000000000,1,"10(0, 0)10000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.000104719226531078,2,13C13C,0.00011757749636802,10001000x00000000xx0x,13C/13C,1000xxx000000000000xx,13C,x000100x0xxx00000000x,13C,10xxxxx0x00000xxxxxxx,13C,x000xxxx0xxx00000xx0x,Unsub,13C Cmethyl | 13C Ccarboxyl
001100000000000000000,1,"00(1, 1)00000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.000105818574566349,2,13C13C,0.000118811831207056,00110000x00000000xx0x,13C/13C,0011xxx000000000000xx,13C/13C,x011000x0xxx00000000x,13C/13C,00xxxxx0x00000xxxxxxx,Unsub,x011xxxx0xxx00000xx0x,13C/13C,13C Calphabeta | 13C Calphabeta
100100000000000000000,2,"10(0, 1)00000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.00020836440971312,2,13C13C,0.000233949069696368,10010000x00000000xx0x,13C/13C,1001xxx000000000000xx,13C/13C,x001000x0xxx00000000x,13C,10xxxxx0x00000xxxxxxx,13C,x001xxxx0xxx00000xx0x,13C,13C Cmethyl | 13C Calphabeta
000110000000000000000,2,"00(0, 1)10000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.000212728062272557,2,13C13C,0.00023884852665336,00011000x00000000xx0x,13C/13C,0001xxx000000000000xx,13C,x001100x0xxx00000000x,13C/13C,00xxxxx0x00000xxxxxxx,Unsub,x001xxxx0xxx00000xx0x,13C,13C Calphabeta | 13C Ccarboxyl
010100000000000000000,2,"01(0, 1)00000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.000210546235992839,2,13C13C,0.000236398798174864,01010000x00000000xx0x,13C/13C,0101xxx000000000000xx,13C/13C,x101000x0xxx00000000x,13C/13C,01xxxxx0x00000xxxxxxx,13C,x101xxxx0xxx00000xx0x,13C/13C,13C Cgamma | 13C Calphabeta
010010000000000000000,1,"01(0, 0)10000(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",0.000105815762934545,2,13C13C,0.00011880867434046,01001000x00000000xx0x,13C/13C,0100xxx000000000000xx,13C,x100100x0xxx00000000x,13C/13C,01xxxxx0x00000xxxxxxx,13C,x100xxxx0xxx00000xx0x,13C,13C Cgamma | 13C Ccarboxyl
000100001000000000000,2,"00(0, 1)00001(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",7.20873290347043e-05,2,13C15N,8.093879174368e-05,00010000x00000000xx0x,13C,0001xxx010000000000xx,13C/15N,x001000x1xxx00000000x,13C/15N,00xxxxx0x00000xxxxxxx,Unsub,x001xxxx1xxx00000xx0x,13C/15N,13C Calphabeta | 15N Namine
100000001000000000000,1,"10(0, 0)00001(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",3.54862882619292e-05,2,13C15N,3.984358047176e-05,10000000x00000000xx0x,13C,1000xxx010000000000xx,13C/15N,x000000x1xxx00000000x,15N,10xxxxx0x00000xxxxxxx,13C,x000xxxx1xxx00000xx0x,15N,13C Cmethyl | 15N Namine
010000001000000000000,1,"01(0, 0)00001(0, 0, 0)(0, 0)(0, 0, 0)(0, 0)00",3.58578724322112e-05,2,13C15N,4.026079073848e-05,01000000x00000000xx0x,13C,0100xxx010000000000xx,13C/15N,x100000x1xxx00000000x,13C/15N,01xxxxx0x00000xxxxxxx,13C,x100xxxx1xxx00000xx0x,13C/15N,13C Cgamma | 15N Namine


In [20]:
Isotopologues[Isotopologues['Precise Identity'] == '13C Calphabeta   |   D Hgamma']

,Number,full,Stochastic,Mass,Composition,Stochastic U,133 Identity,133 Subs,104 Identity,104 Subs,102 Identity,102 Subs,61 Identity,61 Subs,56 Identity,56 Subs,Precise Identity
000100000000010000000,4,"00(0, 1)00000(0, 0, 0)(0, 1)(0, 0, 0)(0, 0)00",5.44365384964338e-06,2,13CD,6.112069501824e-06,00010000x00001000xx0x,13C/D,0001xxx000000100000xx,13C/D,x001000x0xxx01000000x,13C/D,00xxxxx0x00001xxxxxxx,D,x001xxxx0xxx01000xx0x,13C/D,13C Calphabeta | D Hgamma


In [21]:
M2Df

,133,104,102,61,56
34S,0.885385079955042,0.885499742241480,0.000000000000000,0.884346077822152,0.000000000000000
18O,0.077003477155309,0.000000000000000,0.076970767109481,0.000000000000000,0.000000000000000
13C,0.004505859657177,0.010300218360627,0.016438468444087,0.016287975398805,0.020043499856695
13C/D,0.001383269015441,0.001230884649494,0.001102892620213,0.000000000000000,0.000000000000000
13C/33S,0.008469160780368,0.006762191163025,0.000000000000000,0.003359194749207,0.000000000000000
13C/13C,0.023253153436663,0.013896023426708,0.014032741475162,0.002289048476152,0.006989948544597
Unsub,0.000000000000000,0.077259447075703,0.886868776398939,0.087742186021006,0.968301907824042
33S,0.000000000000000,0.001879614453847,0.000000000000000,0.005975517532678,0.000000000000000
13C/15N,0.000000000000000,0.003171878629115,0.003186546205213,0.000000000000000,0.002385004206240
15N,0.000000000000000,0.000000000000000,0.001399807746905,0.000000000000000,0.002279639568427


In [22]:
###In real situations, we can't use the same U^M+N for all fragments, because we will lose low abundance peaks
###corresponding to different isotopologues for each fragment. So for each fragment, we see set X_i of isotopologues
###where X_i is a subset of M+N. We need to calculate an individual U^X_i for each. After using this information
###to calculate site-specific U values, we can perform a linear solve. 

###We could calculate an abundance cutoff (i.e. only peaks with >0.01 percent abundance are included) for fast
###predictive work. Implementing that should be straightforward but not useful here. Instead, I think we need to 
###deal with the headache of a manual list of peaks. 

###At some point, may be able to set this via the input file?
observedPeaks = {'M2':{'133':['34S','18O','13C','13C/D','13C/33S','13C/13C'],
                       '104':['34S','13C','13C/D','13C/33S','13C/13C','Unsub','33S','13C/15N'],
                       '102':['18O','13C','13C/D','13C/13C','Unsub','13C/15N','15N'],
                       '61':['34S','13C','13C/33S','13C/13C','Unsub','33S'],
                       '56':['13C','13C/13C','Unsub','13C/15N','15N']}}

MNKey = "M2"
store = []
for fragment in fragKeys:
    CMatrix = []
    MeasurementVector = []
    IsotopologueFragments = Isotopologues[fragment + ' Subs']
    for sub, v in M2Df[fragment].iteritems():
        #If the observed intensity of a fragment is 0, we do not include it
        if v != 0:
            c = list(IsotopologueFragments.isin([sub]) * 1)
            CMatrix.append(c)
            MeasurementVector.append(v)

    comp = np.array(CMatrix,dtype=float)
    meas = np.array(MeasurementVector,dtype = float)
    AugMatrix = np.column_stack((comp, meas))

    solve = GJElim(AugMatrix, augMatrix = True)

    #Take everything but the final column, which is just the answer
    solution = solve[0][:,:-1]

    MNSolution = checkSolutionIsotopologues(solve, Isotopologues, MNKey)
    
    if UValueReference["M2"][fragment] != None:
        relevantM, relevantU = UValueReference["M2"][fragment][0], UValueReference[MNKey][fragment][1]
        M = MNSolution[MNSolution['Composition'] == relevantM][MNKey + ' Percent Abundance'].sum()
        UMN = relevantU / M
        MNSolution['U' + MNKey] = UMN
        computeMNUValues(MNSolution, MNKey, df)
    
    store.append(MNSolution)

In [23]:
x = store[0]
x[x['Number'] < 5]

,Matrix Row,M2 Percent Abundance,Stochastic U,Composition,Number,UM2,U Values,Deltas,Clumped Deltas
13C Cgamma | 33S Ssulfur & 13C Ccarboxyl | 33S Ssulfur & 13C Calphabeta | 33S Ssulfur & 13C Cmethyl | 33S Ssulfur,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.008469160780368,0.000430053363265,13C33S,4,0.050778745901482,0.000430053363265,N/A,0
18O Ocarboxyl1 & 18O Ocarboxyl2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.077003477155309,0.003910140000000,18O,2,0.050778745901482,0.003910140000000,-25.0000000000006,N/A
34S Ssulfur,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.885385079955042,0.044958744000000,34S,1,0.050778745901482,0.044958744000000,4.85436893203905,N/A


In [24]:
x['calc U'] =  x['Stochastic U'] / x['M2 Percent Abundance']
x

,Matrix Row,M2 Percent Abundance,Stochastic U,Composition,Number,UM2,U Values,Deltas,Clumped Deltas,calc U
13C Cmethyl | 13C Cgamma & 13C Cmethyl | 13C Ccarboxyl & 13C Calphabeta | 13C Calphabeta & 13C Cmethyl | 13C Calphabeta & 13C Calphabeta | 13C Ccarboxyl & 13C Cgamma | 13C Calphabeta & 13C Cgamma | 13C Ccarboxyl,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",0.023253153436663,0.001180765969768,13C13C,7,0.050778745901482,0.001180765969768,N/A,0,0.050778745901482
13C Calphabeta | 15N Namine & 13C Cmethyl | 15N Namine & 13C Cgamma | 15N Namine & 13C Ccarboxyl | 15N Namine & 13C Calphabeta | D Hprotonated & 13C Ccarboxyl | D Hprotonated & 13C Calphabeta | D Hamine & 13C Cgamma | D Hamine & 13C Cmethyl | D Hamine & 13C Cmethyl | D Hprotonated & 13C Cgamma | D Hprotonated & 13C Ccarboxyl | D Hamine,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",0.004505859657177,0.000228801902600,13C15N & 13CD & 13CD & 13CD & 13CD & 13CD & 13...,12,0.050778745901482,0.000228801902600,N/A,0,0.050778745901482
13C Cgamma | 33S Ssulfur & 13C Ccarboxyl | 33S Ssulfur & 13C Calphabeta | 33S Ssulfur & 13C Cmethyl | 33S Ssulfur,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.008469160780368,0.000430053363265,13C33S,4,0.050778745901482,0.000430053363265,N/A,0,0.050778745901482
13C Calphabeta | D Hmethyl & 13C Calphabeta | D Hgamma & 13C Ccarboxyl | D Hmethyl & 13C Calphabeta | D Hhydroxyl & 13C Ccarboxyl | D Hhydroxyl & 13C Ccarboxyl | D Hgamma & 13C Calphabeta | D Halphabeta & 13C Cgamma | D Hhydroxyl & 13C Cmethyl | D Hmethyl & 13C Cmethyl | D Hgamma & 13C Cmethyl | D Halphabeta & 13C Cmethyl | D Hhydroxyl & 13C Cgamma | D Hmethyl & 13C Ccarboxyl | D Halphabeta & 13C Cgamma | D Hgamma & 13C Cgamma | D Halphabeta,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.001383269015441,0.000070240665848,13CD,16,0.050778745901482,0.000070240665848,N/A,0,0.050778745901482
18O Ocarboxyl1 & 18O Ocarboxyl2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.077003477155309,0.003910140000000,18O,2,0.050778745901482,0.003910140000000,-25.0000000000006,N/A,0.050778745901482
34S Ssulfur,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.885385079955042,0.044958744000000,34S,1,0.050778745901482,0.044958744000000,4.85436893203905,N/A,0.050778745901482


In [65]:
CMatrix = []
MeasurementVector = []
for fragSol in store:
    if 'U' + MNKey in fragSol:
        for i, v in fragSol.iterrows():
            CMatrix.append(v['Matrix Row'])
            MeasurementVector.append(v['U Values'])
        

In [66]:
#Construct augmented matrix
comp = np.array(CMatrix,dtype=float)
meas = np.array(MeasurementVector,dtype = float)
AugMatrix = np.column_stack((comp, meas))

solve = GJElim(AugMatrix, augMatrix = True)

In [67]:
MNSolution = checkSolutionIsotopologues(solve, Isotopologues, MNKey, fullSolve = True)
MNSolution

,U Values,Stochastic U,Composition,Number
13C Cmethyl | 13C Cgamma,0.000120622570043,0.000116371573328,13C13C,1
13C Cmethyl | 13C Ccarboxyl & 13C Calphabeta | 13C Ccarboxyl & 13C Cgamma | 13C Ccarboxyl,0.000482490280170,0.000475234697362,13C13C,3
13C Calphabeta | 13C Calphabeta & 13C Calphabeta | 13C Ccarboxyl & MINUS -1 13C Cgamma | 15N Namine & MINUS -1 13C Cgamma | 17O Ocarboxyl2 & MINUS -1 13C Cmethyl | 17O Ocarboxyl1 & MINUS -1 13C Cmethyl | 17O Ocarboxyl2 & MINUS -1 13C Cgamma | 17O Ocarboxyl1 & MINUS -1 13C Cgamma | D Hamine & MINUS -1 13C Cgamma | D Hhydroxyl & MINUS -1 13C Cmethyl | D Halphabeta & MINUS -1 13C Cmethyl | D Hamine & MINUS -1 13C Cmethyl | D Hhydroxyl & MINUS -1 13C Cmethyl | D Hprotonated & MINUS -1 13C Cgamma | D Hprotonated & MINUS -1 13C Cgamma | D Halphabeta & 15N Namine | D Hmethyl & 15N Namine | D Hprotonated & MINUS -1 33S Ssulfur | D Hamine & MINUS -1 33S Ssulfur | D Halphabeta,0.000274644552953,0.000271651742842,13C13C & 13C15N & 13C17O & 13C17O & 13C17O & 1...,19
13C Cmethyl | 13C Calphabeta & MINUS -1 13C Calphabeta | 13C Ccarboxyl & MINUS -1 13C Cgamma | 13C Ccarboxyl,-0.000099120285818,-0.000123708131297,13C13C,3
13C Cgamma | 13C Calphabeta & 13C Cgamma | 13C Ccarboxyl & 13C Cgamma | 15N Namine & 13C Cgamma | 17O Ocarboxyl2 & 13C Cmethyl | 17O Ocarboxyl1 & 13C Cmethyl | 17O Ocarboxyl2 & 13C Cgamma | 17O Ocarboxyl1 & 13C Cgamma | D Hamine & 13C Cgamma | D Hhydroxyl & 13C Cmethyl | D Halphabeta & 13C Cmethyl | D Hamine & 13C Cmethyl | D Hhydroxyl & 13C Cmethyl | D Hprotonated & 13C Cgamma | D Hprotonated & 13C Cgamma | D Halphabeta & MINUS -1 15N Namine | D Hmethyl & MINUS -1 15N Namine | D Hprotonated & 33S Ssulfur | D Hamine & 33S Ssulfur | D Halphabeta,0.000427588583077,0.000441216087534,13C13C & 13C15N & 13C17O & 13C17O & 13C17O & 1...,19
13C Calphabeta | 15N Namine & 13C Cgamma | 15N Namine & MINUS -1 15N Namine | D Hmethyl & MINUS -1 15N Namine | D Hprotonated & 33S Ssulfur | D Hamine & 33S Ssulfur | D Halphabeta,0.000127427047402,0.000125731127785,13C15N & 15ND & 15ND & 33SD & 33SD,6
13C Cmethyl | 15N Namine & 15N Namine | D Hmethyl & 15N Namine | D Hprotonated & MINUS -1 33S Ssulfur | D Hamine & MINUS -1 33S Ssulfur | D Halphabeta,0.000034455990339,0.000035312035168,13C15N & 15ND & 15ND & 33SD & 33SD,5
13C Ccarboxyl | 15N Namine & 15N Namine | D Hmethyl & 15N Namine | D Hprotonated & MINUS -1 33S Ssulfur | D Hamine & MINUS -1 33S Ssulfur | D Halphabeta,0.000029646421826,0.000036146455702,13C15N & 15ND & 15ND & 33SD & 33SD,5
13C Calphabeta | 17O Ocarboxyl2 & 13C Cgamma | 17O Ocarboxyl2 & 13C Calphabeta | 17O Ocarboxyl1 & 13C Cmethyl | 17O Ocarboxyl1 & 13C Cmethyl | 17O Ocarboxyl2 & 13C Cgamma | 17O Ocarboxyl1 & 13C Calphabeta | D Hhydroxyl & MINUS -1 13C Ccarboxyl | D Hprotonated & 13C Calphabeta | D Halphabeta & 13C Cgamma | D Hhydroxyl & 13C Cmethyl | D Halphabeta & 13C Cmethyl | D Hhydroxyl & 13C Ccarboxyl | D Halphabeta & 13C Cgamma | D Halphabeta & 15N Namine | D Hamine & 15N Namine | D Halphabeta & 15N Namine | D Hhydroxyl & 15N Namine | D Hprotonated & 17O Ocarboxyl2 | 15N Namine & 17O Ocarboxyl1 | 15N Namine & 17O Ocarboxyl2 | D Hamine & 17O Ocarboxyl2 | D Halphabeta & 17O Ocarboxyl1 | D Hamine & 17O Ocarboxyl1 | D Halphabeta & D Hamine | D Hprotonated & D Hamine | D Hhydroxyl & D Hamine | D Hamine & D Halphabeta | D Hprotonated & D Halphabeta | D Hhydroxyl & D Halphabeta | D Hamine & D Halphabeta | D Halphabeta,0.000072243168584,0.000072901342659,13C17O & 13CD & 13CD & 13CD & 13CD & 13CD & 13...,31
13C Ccarboxyl | 17O Ocarboxyl1 & 13C Ccarboxyl | 17O Ocarboxyl2 & 13C Ccarboxyl | D Hhydroxyl & 13C Ccarboxyl | D Hprotonated & 17O Ocarboxyl1 | 17O Ocarboxyl2 & 17O Ocarboxyl2 | D Hprotonated & 17O Ocarboxyl2 | D Hhydroxyl & 17O Ocarboxyl1 | D Hprotonated & 17O Ocarboxyl1 | D Hhydroxyl & D Hhydroxyl | D Hprotonated,0.000012201895616,0.000012489829192,13C17O & 13CD & 13CD & 17O17O & 17OD & 17OD & ...,10


In [68]:
M2Solution = computeMNUValues(MNSolution, MNKey, df, applyUMN = False)

In [69]:
M2Solution[M2Solution['Number'] < 5]

,U Values,Stochastic U,Composition,Number,Deltas,Clumped Deltas
13C Cmethyl | 13C Cgamma,0.000120622570043,0.000116371573328,13C13C,1,N/A,36.5295113973574
13C Cmethyl | 13C Ccarboxyl & 13C Calphabeta | 13C Ccarboxyl & 13C Cgamma | 13C Ccarboxyl,0.000482490280170,0.000475234697362,13C13C,3,N/A,15.2673675738213
13C Cmethyl | 13C Calphabeta & MINUS -1 13C Calphabeta | 13C Ccarboxyl & MINUS -1 13C Cgamma | 13C Ccarboxyl,-0.000099120285818,-0.000123708131297,13C13C,3,N/A,-198.756906453769
13C Ccarboxyl | 33S Ssulfur,0.000086925906728,0.000086722239748,13C33S,1,N/A,2.34849768781786
13C Calphabeta | 33S Ssulfur,0.000173851813456,0.000172555020628,13C33S,1,N/A,7.51524252126079
17O Ocarboxyl1 | 33S Ssulfur & 17O Ocarboxyl2 | 33S Ssulfur & 33S Ssulfur | D Hprotonated & 33S Ssulfur | D Hhydroxyl,0.000008499741463,0.000008709866008,17O33S & 33SD & 33SD,4,N/A,-24.1248884316095
18O Ocarboxyl1 & 18O Ocarboxyl2,0.004029867961165,0.003910140000000,18O,2,N/A,30.6198655713217
33S Ssulfur | 15N Namine & 33S Ssulfur | D Hamine & 33S Ssulfur | D Halphabeta,0.000035343919901,0.000035798653591,33S15N & 33SD & 33SD,3,N/A,-12.7025361111441
34S Ssulfur,0.044958744000000,0.044958744000000,34S,1,4.85436893203905,N/A


In [29]:
M1Measurements = {}
M2Measurements = {}
M3Measurements = {}
M4Measurements = {}

for i, v in inputData.iterrows():
    if len(v['Unnamed: 0'].split(' ')) > 2:
        frag = v['Unnamed: 0'].split(' ')[1]
        sub = v['Unnamed: 0'].split(' ')[2]
        if sub == '':
            sub = 'Unsub'

        if v['Unnamed: 0'][:2] == 'M1':
            if frag not in M1Measurements:
                M1Measurements[frag] = {}
            M1Measurements[frag][sub] = v['Adj. Rel. Abundance']
            
        if v['Unnamed: 0'][:2] == 'M2':
            if frag not in M2Measurements:
                M2Measurements[frag] = {}
            M2Measurements[frag][sub] = v['Adj. Rel. Abundance']
            
        if v['Unnamed: 0'][:2] == 'M3':
            if frag not in M3Measurements:
                M3Measurements[frag] = {}
            M3Measurements[frag][sub] = v['Adj. Rel. Abundance']
            
        if v['Unnamed: 0'][:2] == 'M4':
            if frag not in M4Measurements:
                M4Measurements[frag] = {}
            M4Measurements[frag][sub] = v['Adj. Rel. Abundance']
            
            
M1Df = pd.DataFrame.from_dict(M1Measurements)
M1Df.fillna(0, inplace = True)
M2Df = pd.DataFrame.from_dict(M2Measurements)
M2Df.fillna(0, inplace = True)
M3Df = pd.DataFrame.from_dict(M3Measurements)
M3Df.fillna(0, inplace = True)
M4Df = pd.DataFrame.from_dict(M4Measurements)
M4Df.fillna(0, inplace = True)

MNDfList = [M1Df, M2Df, M3Df, M4Df]

In [30]:
fragments = [frag_133, frag_104, frag_102, frag_61, frag_56]
fragKeys = ['133','104','102','61','56']

In [31]:
UnsubConc = M0['000000000000000000000']['Conc']

#For each fragment we will observe
massSelected = [M1, M2, M3, M4]
massKeys = ['M1','M2','M3','M4']

for selection in massSelected:
#For each fragment we will observe
    for j, fragment in enumerate(fragments):
        #compute the isotopologues present after fragmentation and track their concentrations
        fragmentedDict = {}
        for isotopologue, value in selection.items():
            value['Stochastic U'] = value['Conc'] / UnsubConc
            newIsotopologue = expandAndFragment(fragment, df['Number'].values, isotopologue)
            selection[isotopologue][fragKeys[j] + ' Identity'] = newIsotopologue

            sub = computeSubs(newIsotopologue, siteElements)

            if sub == '':
                sub = 'Unsub'

            selection[isotopologue][fragKeys[j] + ' Subs'] = sub

In [32]:
elIDs = df['IDS'].values
numberAtSite = df['Number'].values

siteListN = [(x,y) for x,y in zip(df.index, numberAtSite)]
siteNameList = []
for site in siteListN:
    siteNameList += [site[0]] * site[1]

In [33]:
def filterEmptyStr(string):
    if string == '':
        return False
    else:
        return True
    
IsotopologuesBySelection = {}

for j, selection in enumerate(massSelected):
    Isotopologues = pd.DataFrame.from_dict(selection).T
    Isotopologues.rename(columns={'Conc':'Stochastic',"Subs": "Composition"},inplace = True)
    preciseStrings = []

    for i, v in Isotopologues.iterrows():
        index = list(Isotopologues.index).index(i)
        Subs = [uEl(element, int(number)) for element, number in zip(siteElements, i)]
        Precise = [x + " " + y for x, y in zip(Subs, siteNameList) if x != '']
        output = '   |   '.join(Precise)
        preciseStrings.append(output)
    Isotopologues['Precise Identity'] = preciseStrings
    Isotopologues.sort_values('Composition',inplace = True)
    
    IsotopologuesBySelection[massKeys[j]] = Isotopologues

In [34]:
solved = {}
for index, selection in enumerate(massSelected):
    Isotopologues = IsotopologuesBySelection[massKeys[index]]
    MNDf = MNDfList[index]
    

    MNKey = massKeys[index]
    store = []
    for fragment in fragKeys:
        CMatrix = []
        MeasurementVector = []
        IsotopologueFragments = Isotopologues[fragment + ' Subs']
        for sub, v in MNDf[fragment].iteritems():
            #If the observed intensity of a fragment is 0, we do not include it
            if v != 0:
                c = list(IsotopologueFragments.isin([sub]) * 1)
                CMatrix.append(c)
                MeasurementVector.append(v)

        comp = np.array(CMatrix,dtype=float)
        meas = np.array(MeasurementVector,dtype = float)
        AugMatrix = np.column_stack((comp, meas))

        solve = GJElim(AugMatrix, augMatrix = True)

        #Take everything but the final column, which is just the answer
        solution = solve[0][:,:-1]

        MNSolution = checkSolutionIsotopologues(solve, Isotopologues, MNKey)
        MNSolution['Fragment'] = fragment

        if UValueReference[MNKey][fragment] != None:
            relevantM, relevantU = UValueReference[MNKey][fragment][0], UValueReference[MNKey][fragment][1]
            M = MNSolution[MNSolution['Composition'] == relevantM][MNKey + ' Percent Abundance'].sum()
            UMN = relevantU / M
            MNSolution['U' + MNKey] = UMN
            computeMNUValues(MNSolution, MNKey, df)

        store.append(MNSolution)
        
    CMatrix = []
    MeasurementVector = []
    for fragSol in store:
        if 'U' + MNKey in fragSol:
            for i, v in fragSol.iterrows():
                CMatrix.append(v['Matrix Row'])
                MeasurementVector.append(v['U Values'])
                
    #Construct augmented matrix
    comp = np.array(CMatrix,dtype=float)
    meas = np.array(MeasurementVector,dtype = float)
    AugMatrix = np.column_stack((comp, meas))

    solve = GJElim(AugMatrix, augMatrix = True)
        
    MNSolution = checkSolutionIsotopologues(solve, Isotopologues, MNKey, fullSolve = True)
    
    MNSolution = computeMNUValues(MNSolution, MNKey, df, applyUMN = False)
    solved[MNKey] = (MNSolution)

In [35]:
solved['M2']

,U Values,Stochastic U,Composition,Number,Deltas,Clumped Deltas
13C Cmethyl | 13C Cgamma,0.000120622570043,0.000116371573328,13C13C,1,N/A,36.5295113973574
13C Cmethyl | 13C Ccarboxyl & 13C Calphabeta | 13C Ccarboxyl & 13C Cgamma | 13C Ccarboxyl,0.000482490280170,0.000475234697362,13C13C,3,N/A,15.2673675738213
13C Calphabeta | 13C Calphabeta & 13C Calphabeta | 13C Ccarboxyl & MINUS -1 13C Cgamma | 15N Namine & MINUS -1 13C Cgamma | 17O Ocarboxyl2 & MINUS -1 13C Cmethyl | 17O Ocarboxyl1 & MINUS -1 13C Cmethyl | 17O Ocarboxyl2 & MINUS -1 13C Cgamma | 17O Ocarboxyl1 & MINUS -1 13C Cgamma | D Hamine & MINUS -1 13C Cgamma | D Hhydroxyl & MINUS -1 13C Cmethyl | D Halphabeta & MINUS -1 13C Cmethyl | D Hamine & MINUS -1 13C Cmethyl | D Hhydroxyl & MINUS -1 13C Cmethyl | D Hprotonated & MINUS -1 13C Cgamma | D Hprotonated & MINUS -1 13C Cgamma | D Halphabeta & 15N Namine | D Hmethyl & 15N Namine | D Hprotonated & MINUS -1 33S Ssulfur | D Hamine & MINUS -1 33S Ssulfur | D Halphabeta,0.000274644552953,0.000271651742842,13C13C & 13C15N & 13C17O & 13C17O & 13C17O & 1...,19,N/A,11.0170841539705
13C Cmethyl | 13C Calphabeta & MINUS -1 13C Calphabeta | 13C Ccarboxyl & MINUS -1 13C Cgamma | 13C Ccarboxyl,-0.000099120285818,-0.000123708131297,13C13C,3,N/A,-198.756906453769
13C Cgamma | 13C Calphabeta & 13C Cgamma | 13C Ccarboxyl & 13C Cgamma | 15N Namine & 13C Cgamma | 17O Ocarboxyl2 & 13C Cmethyl | 17O Ocarboxyl1 & 13C Cmethyl | 17O Ocarboxyl2 & 13C Cgamma | 17O Ocarboxyl1 & 13C Cgamma | D Hamine & 13C Cgamma | D Hhydroxyl & 13C Cmethyl | D Halphabeta & 13C Cmethyl | D Hamine & 13C Cmethyl | D Hhydroxyl & 13C Cmethyl | D Hprotonated & 13C Cgamma | D Hprotonated & 13C Cgamma | D Halphabeta & MINUS -1 15N Namine | D Hmethyl & MINUS -1 15N Namine | D Hprotonated & 33S Ssulfur | D Hamine & 33S Ssulfur | D Halphabeta,0.000427588583077,0.000441216087534,13C13C & 13C15N & 13C17O & 13C17O & 13C17O & 1...,19,N/A,-30.8862365669921
13C Calphabeta | 15N Namine & 13C Cgamma | 15N Namine & MINUS -1 15N Namine | D Hmethyl & MINUS -1 15N Namine | D Hprotonated & 33S Ssulfur | D Hamine & 33S Ssulfur | D Halphabeta,0.000127427047402,0.000125731127785,13C15N & 15ND & 15ND & 33SD & 33SD,6,N/A,13.4884626108569
13C Cmethyl | 15N Namine & 15N Namine | D Hmethyl & 15N Namine | D Hprotonated & MINUS -1 33S Ssulfur | D Hamine & MINUS -1 33S Ssulfur | D Halphabeta,0.000034455990339,0.000035312035168,13C15N & 15ND & 15ND & 33SD & 33SD,5,N/A,-24.2422965826666
13C Ccarboxyl | 15N Namine & 15N Namine | D Hmethyl & 15N Namine | D Hprotonated & MINUS -1 33S Ssulfur | D Hamine & MINUS -1 33S Ssulfur | D Halphabeta,0.000029646421826,0.000036146455702,13C15N & 15ND & 15ND & 33SD & 33SD,5,N/A,-179.824930253597
13C Calphabeta | 17O Ocarboxyl2 & 13C Cgamma | 17O Ocarboxyl2 & 13C Calphabeta | 17O Ocarboxyl1 & 13C Cmethyl | 17O Ocarboxyl1 & 13C Cmethyl | 17O Ocarboxyl2 & 13C Cgamma | 17O Ocarboxyl1 & 13C Calphabeta | D Hhydroxyl & MINUS -1 13C Ccarboxyl | D Hprotonated & 13C Calphabeta | D Halphabeta & 13C Cgamma | D Hhydroxyl & 13C Cmethyl | D Halphabeta & 13C Cmethyl | D Hhydroxyl & 13C Ccarboxyl | D Halphabeta & 13C Cgamma | D Halphabeta & 15N Namine | D Hamine & 15N Namine | D Halphabeta & 15N Namine | D Hhydroxyl & 15N Namine | D Hprotonated & 17O Ocarboxyl2 | 15N Namine & 17O Ocarboxyl1 | 15N Namine & 17O Ocarboxyl2 | D Hamine & 17O Ocarboxyl2 | D Halphabeta & 17O Ocarboxyl1 | D Hamine & 17O Ocarboxyl1 | D Halphabeta & D Hamine | D Hprotonated & D Hamine | D Hhydroxyl & D Hamine | D Hamine & D Halphabeta | D Hprotonated & D Halphabeta | D Hhydroxyl & D Halphabeta | D Hamine & D Halphabeta | D Halphabeta,0.000072243168584,0.000072901342659,13C17O & 13CD & 13CD & 13CD & 13CD & 13CD & 13...,31,N/A,-9.02828467495009
13C Ccarboxyl | 17O Ocarboxyl1 & 13C Ccarboxyl | 17O Ocarboxyl2 & 13C Ccarboxyl | D Hhydroxyl & 13C Ccarboxyl | D Hprotonated & 17O Ocarboxyl1 | 17O Ocarboxyl2 & 17O Ocarboxyl2 | D Hprotonated & 17O Ocarboxyl2 | D Hhydroxyl & 17O Ocarboxyl1 | D Hprotonated & 17O Ocarboxy

In [36]:
M1Solution = solved['M1']
M2Solution = solved['M2']
M3Solution = solved['M3']
M4Solution = solved['M4']

In [37]:
M4Solution[M4Solution['Number'] < 5][['Number','Deltas','Clumped Deltas']]

,Number,Deltas,Clumped Deltas
13C Cmethyl | 13C Ccarboxyl | 34S Ssulfur & 13C Calphabeta | 13C Ccarboxyl | 34S Ssulfur & 13C Cgamma | 13C Ccarboxyl | 34S Ssulfur,3,N/A,-59.3742960090154
13C Cmethyl | 13C Cgamma | 34S Ssulfur,1,N/A,36.0070012427078
13C Cmethyl | 34S Ssulfur | 15N Namine & 13C Cgamma | 34S Ssulfur | 15N Namine & 13C Calphabeta | 34S Ssulfur | 15N Namine,3,N/A,4.70849066949608
13C Cgamma | 34S Ssulfur | D Hgamma & 13C Cgamma | 34S Ssulfur | D Hmethyl & 13C Cmethyl | 34S Ssulfur | D Hmethyl & 13C Cmethyl | 34S Ssulfur | D Hgamma,4,N/A,259.309581315639
18O Ocarboxyl1 | 18O Ocarboxyl2,1,N/A,2.61621835306869e-08
18O Ocarboxyl1 | 34S Ssulfur & 18O Ocarboxyl2 | 34S Ssulfur,2,N/A,-9.10493902495091e-10
36S Ssulfur,1,9.22330096700041,N/A


In [38]:
M3Solution[M3Solution['Number'] <4][['Number','Deltas','Clumped Deltas']]

,Number,Deltas,Clumped Deltas
13C Cmethyl | 13C Cgamma | 13C Calphabeta & 13C Cgamma | 13C Calphabeta | 13C Calphabeta & 13C Cmethyl | 13C Calphabeta | 13C Calphabeta,3,N/A,46.824613206055908
13C Cgamma | 13C Ccarboxyl | 33S Ssulfur & 13C Calphabeta | 13C Ccarboxyl | 33S Ssulfur & 13C Cmethyl | 13C Ccarboxyl | 33S Ssulfur,3,N/A,-23.229566303873071
13C Ccarboxyl | 34S Ssulfur,1,N/A,-81.025641015653576
13C Calphabeta | 34S Ssulfur & 13C Cmethyl | 34S Ssulfur & 13C Cgamma | 34S Ssulfur,3,N/A,20.466321240832251
17O Ocarboxyl2 | 34S Ssulfur & 17O Ocarboxyl1 | 34S Ssulfur,2,N/A,4.858026134486826
18O Ocarboxyl2 | 33S Ssulfur & 18O Ocarboxyl1 | 33S Ssulfur,2,N/A,19.614461290194729
34S Ssulfur | 15N Namine,1,N/A,-0.000000002636447
34S Ssulfur | D Hprotonated & 34S Ssulfur | D Hamine,2,N/A,-204.457465773675949
34S Ssulfur | D Hhydroxyl & MINUS -1 34S Ssulfur | D Hamine,2,N/A,-7279.871190829378065


In [39]:
M2Solution[M2Solution['Number'] < 5][['Number','Deltas','Clumped Deltas']]

,Number,Deltas,Clumped Deltas
13C Cmethyl | 13C Cgamma,1,N/A,36.5295113973574
13C Cmethyl | 13C Ccarboxyl & 13C Calphabeta | 13C Ccarboxyl & 13C Cgamma | 13C Ccarboxyl,3,N/A,15.2673675738213
13C Cmethyl | 13C Calphabeta & MINUS -1 13C Calphabeta | 13C Ccarboxyl & MINUS -1 13C Cgamma | 13C Ccarboxyl,3,N/A,-198.756906453769
13C Ccarboxyl | 33S Ssulfur,1,N/A,2.34849768781786
13C Calphabeta | 33S Ssulfur,1,N/A,7.51524252126079
17O Ocarboxyl1 | 33S Ssulfur & 17O Ocarboxyl2 | 33S Ssulfur & 33S Ssulfur | D Hprotonated & 33S Ssulfur | D Hhydroxyl,4,N/A,-24.1248884316095
18O Ocarboxyl1 & 18O Ocarboxyl2,2,N/A,30.6198655713217
33S Ssulfur | 15N Namine & 33S Ssulfur | D Hamine & 33S Ssulfur | D Halphabeta,3,N/A,-12.7025361111441
34S Ssulfur,1,4.85436893203905,N/A


In [40]:
M1Solution[M1Solution['Number'] < 2][['Number','Deltas','Clumped Deltas']]

,Number,Deltas,Clumped Deltas
13C Ccarboxyl,1,-74.7500000000009,N/A
13C Calphabeta,1,-22.5624999999999,N/A
13C Cgamma,1,58.8750000000011,N/A
13C Cmethyl,1,-104,N/A
15N Namine,1,2.50000000000017,N/A
33S Ssulfur,1,2.49999999999995,N/A
D Hprotonated,1,-480.294350776381,N/A
D Hhydroxyl,1,104.187388059244,N/A
D Hamine,1,101.516770952699,N/A
D Halphabeta,1,-63.5111806350824,N/A
